# <center>ECON 725: Computer Programming and Data Management in Economics <a class="tocSkip"></center>    
# <center>Python - Introduction to SQLite  <a class="tocSkip"></center>

## Learning Objectives
<hr>

- Understand the basics of SQL Lite
- Learn how to connect to a SQL Lite database
- Learn how to create tables in a SQL Lite database
- Learn how to insert data into a SQL Lite database
- Learn how to query data from a SQL Lite database
- Learn how to update data in a SQL Lite database
- Learn how to delete data from a SQL Lite database


## Introduction
<hr>

When you load a CSV file in Python and assign it to a variable, it consumes your computer's memory. Accessing data from a SQL database is more efficient and enables you to import only the necessary portions of the data.

In this lesson, we will explore some methods to achieve this. This is our first introduction to SQL databases. We will use the SQLite database, which is a lightweight database that is easy to set up and use. SQLite is a self-contained, serverless, zero-configuration, transactional SQL database engine. It is the most widely deployed database in the world with applications in many devices and operating systems.

### What is SQL?
SQL stands for Structured Query Language. It is a standard language for storing, manipulating, and retrieving data in databases. SQL is used to communicate with a database. According to ANSI (American National Standards Institute), it is the standard language for relational database management systems. SQL statements are used to perform tasks such as update data on a database, or retrieve data from a database. This is our first introduction to SQL databases. We are going to explore more about this topic throughout the course as we progress.

### The sqlite3 module
The [sqlite3](https://docs.python.org/3/library/sqlite3.html) module provides a straightforward interface for interacting with SQLite databases. A connection object is created using `sqlite3.connect()`; the connection must be closed at the end of the session with the `.close()` command. While the connection is open, any interactions with the database require you to make a cursor object with the `.cursor()` command. The cursor is then ready to perform all kinds of operations with `.execute()`.

In [1]:
import sqlite3
import pandas as pd

In [ ]:
# create empty database
connection = sqlite3.connect("gta.db")
# communicate with the database
cursor = connection.cursor()

In [12]:
# data to be inserted into the database
release_list = [
    (1997, "Grand Theft Auto", "state of New Guernsey"),
    (1999, "Grand Theft Auto 2", "Anywhere, USA"),
    (2001, "Grand Theft Auto III", "Liberty City"),
    (2002, "Grand Theft Auto: Vice City", "Vice City"),
    (2004, "Grand Theft Auto: San Andreas", "state of San Andreas"),
    (2008, "Grand Theft Auto IV", "Liberty City"),
    (2013, "Grand Theft Auto V", "Los Santos")
]

In [13]:
city_list = [
    ("Liberty City", "New York"),
    ("state of New Guernsey", "state of New Jersey"),
    ("Anywhere, USA", "all USA cities"),
    ("Vice City", "Miami"),
    ("state of San Andreas", "state of California"),
    ("Los Santos", "Los Angeles")
]

In [14]:
# create database table and populate it with release_list
cursor.execute("create table gta (release_year integer, release_name text, city text)")
cursor.executemany("insert into gta values (?,?,?)", release_list)
# save changes immediatley
connection.commit()

In [15]:
# print all the rows from the gta table
for row in cursor.execute("select * from gta"):
    print(row)

(1997, 'Grand Theft Auto', 'state of New Guernsey')
(1999, 'Grand Theft Auto 2', 'Anywhere, USA')
(2001, 'Grand Theft Auto III', 'Liberty City')
(2002, 'Grand Theft Auto: Vice City', 'Vice City')
(2004, 'Grand Theft Auto: San Andreas', 'state of San Andreas')
(2008, 'Grand Theft Auto IV', 'Liberty City')
(2013, 'Grand Theft Auto V', 'Los Santos')


In [16]:
# print specific rows from the gta table
print("******************************")
cursor.execute("select * from gta where city=:c", {"c": "Liberty City"})
gta_search = cursor.fetchall()
print(gta_search)

******************************
[(2001, 'Grand Theft Auto III', 'Liberty City'), (2008, 'Grand Theft Auto IV', 'Liberty City')]


In [19]:
type(gta_search)

list

In [12]:
# print the columns of the gta table
cursor.execute("PRAGMA table_info(gta)") # special command that retrieves metadata about the columns of a table
columns = cursor.fetchall()
print(columns)

[(0, 'release_year', 'INTEGER', 0, None, 0), (1, 'release_name', 'TEXT', 0, None, 0), (2, 'city', 'TEXT', 0, None, 0)]


In [17]:
# create a new cities table
cursor.execute("create table cities (gta_city text, real_city text)")
cursor.executemany("insert into cities values (?,?)", city_list)
# save changes immediatley
connection.commit()

In [18]:
# print all the rows from the gta table
for row in cursor.execute("select * from cities"):
    print(row)

('Liberty City', 'New York')
('state of New Guernsey', 'state of New Jersey')
('Anywhere, USA', 'all USA cities')
('Vice City', 'Miami')
('state of San Andreas', 'state of California')
('Los Santos', 'Los Angeles')


In [8]:
# select only the rows where gta_city=Liberty City
cursor.execute("select * from cities where gta_city=:c", {"c": "Liberty City"})
cities_search = cursor.fetchall()
print(cities_search)

[('Liberty City', 'New York')]


In [20]:
gta_search

[(2001, 'Grand Theft Auto III', 'Liberty City'),
 (2008, 'Grand Theft Auto IV', 'Liberty City')]

In [10]:
# combine data from 2 tables and manipulate it
print("***************************************")
for i in gta_search:
    #replace every instance of "Liberty City" with "New York"
    adjusted = [cities_search[0][1] if value==cities_search[0][0] else value for value in i]
    print(adjusted)

***************************************
[2001, 'Grand Theft Auto III', 'New York']
[2008, 'Grand Theft Auto IV', 'New York']


## Relational Databases
<hr>


* A relational database organizes its data into *tables*, sometimes called *relations*.
* Each table is made up of a fixed number of *columns* and zero or more *rows*
* Each column has a name, and generally has a data type (SQLite is a bit lenient on data types, but a good database design keeps a uniform type on every row in a column)
* A table usually has a *primary key*—a column or set of columns that is used to uniquely identify a row. Each row primary key must be unique througout that table.

So let's get started by importing some data. Let's say we have a CSV file that holds students and their grades in certain courses:

```
studentid,course,grade,credits
1,PY205,3.8,3
1,PY411,3.2,4
1,PY412,3.1,4
1,E101,3.8,2
1,CS216,3.5,3
2,PY205,3.1,3
2,PY411,2.8,4
2,PY412,2.4,4
2,E101,2.9,2
2,CS216,3.3,3
```
Both students took the same courses. Student 1 did pretty well, student 2 not as well. Grades are assigned a numerical value with A being a 4, B being a 3, etc.

If this data is in a file called ``grades.csv`` then we can import it with the following commands:

In [2]:
# import grades.csv  in a sqlite database

# Load the CSV data
csv_file_path = 'grades.csv'
data = pd.read_csv(csv_file_path)

In [5]:
data.head(10)

,studentid,course,grade,credits
0,1,PY205,3.8,3
1,1,PY411,3.2,4
2,1,PY412,3.1,4
3,1,E101,3.8,2
4,1,CS216,3.5,3
5,2,PY205,3.1,3
6,2,PY411,2.8,4
7,2,PY412,2.4,4
8,2,E101,2.9,2
9,2,CS216,3.3,3


In [4]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('students_grades.db')

# Save the DataFrame to an SQLite table
data.to_sql('grades', conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data imported into the SQLite database successfully!")

Data imported into the SQLite database successfully!


Let's start exploring our new database.     

Check out the [SELECT documentation](https://www.sqlite.org/lang_select.html) and take a look at the syntax flowchart. It's not important to understand it all now, but keep that flowchart handy as we learn more concepts.

The important bits to notice are the different clauses:
* **SELECT** - lists the columns to return in the results, or ``*`` for all columns. These can actually be any expression, which you can use to compute values, as we'll see in a bit.
* **FROM** - defines which tables to use as input to the query. If you have more than one table, then the tables are joined. More on joins later.
* **WHERE** - Defines conditional expressions that filter the rows from the input tables
* **GROUP BY** - Used in aggregate queries. More on this later.
* **HAVING** - Expressions used to filter groups on an aggregate query.
* **ORDER BY** - Expressions used to sort the results before returning them
* **LIMIT** - Places limits or returns a slice of the results.

In [ ]:
# Install this kernel to use the sqlite3 module
# !pip install ipython-sql

Once installed, you can load the SQL extension and connect it to your SQLite database:

In [7]:
# Load the sql extension
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///students_grades.db

Now, you can write SQL queries directly in the notebook using %sql or %%sql magic commands:

In [10]:
# Example 1: Display the first few rows from the grades table
%sql SELECT * FROM grades LIMIT 5;

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,PY411,3.2,4
1,PY412,3.1,4
1,E101,3.8,2
1,CS216,3.5,3


In [11]:
# Example 2: Find all students who have a grade higher than 3.5
%sql SELECT * FROM grades WHERE grade > 3.5;

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,E101,3.8,2


•	Use %sql for a single-line SQL query.

•	Use %%sql for multi-line queries. Example:

In [12]:
%%sql
SELECT course, AVG(grade) as avg_grade
FROM grades
GROUP BY course;

 * sqlite:///students_grades.db
Done.


course,avg_grade
CS216,3.4
E101,3.3499999999999996
PY205,3.45
PY411,3.0
PY412,2.75


#### Filtering rows with a WHERE clause
You can filter rows using a WHERE clause in your query. You can use simple comparison expressions here. You can have multiple conditions separated by ``AND`` and ``OR`` keywords.

In [13]:
%sql SELECT * FROM grades WHERE studentid=1;

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,PY411,3.2,4
1,PY412,3.1,4
1,E101,3.8,2
1,CS216,3.5,3


In [14]:
%sql SELECT * FROM grades WHERE course='PY205';

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
2,PY205,3.1,3


In [15]:
%sql SELECT * FROM grades WHERE studentid=2 AND course='E101';

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
2,E101,2.9,2


In [16]:
%sql SELECT * FROM grades WHERE grade>3.5;

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,E101,3.8,2


#### Expressions and the SELECT clause
You can list specific columns you want in your results in the SELECT clause, seprated by columns. Actually, you can put any expression you want in there, computing results from multiple columns.

From the [SQLite documentation](https://www.sqlite.org/lang_select.html#resultset): 

> The list of expressions between the SELECT and FROM keywords is known as the result expression list. If a result expression is the special expression "*" then all columns in the input data are substituted for that one expression.

In [17]:
%sql SELECT studentid, grade FROM grades;

 * sqlite:///students_grades.db
Done.


studentid,grade
1,3.8
1,3.2
1,3.1
1,3.8
1,3.5
2,3.1
2,2.8
2,2.4
2,2.9
2,3.3


This example shows computing a column from an expression. The grade points earned for a course is the grade earned times the credits. This quantity is used in computing grade point averages.

In [18]:
%sql SELECT studentid, course, grade*credits FROM grades;

 * sqlite:///students_grades.db
Done.


studentid,course,grade*credits
1,PY205,11.399999999999999
1,PY411,12.8
1,PY412,12.4
1,E101,7.6
1,CS216,10.5
2,PY205,9.3
2,PY411,11.2
2,PY412,9.6
2,E101,5.8
2,CS216,9.899999999999999


You can change the result column name by giving an alternative name like so. This is useful for clarity, but also you can refer to these names later in the query in the WHERE clause and a few other clauses.

In [19]:
%sql SELECT studentid, course, grade*credits AS gradepoints FROM grades;

 * sqlite:///students_grades.db
Done.


studentid,course,gradepoints
1,PY205,11.399999999999999
1,PY411,12.8
1,PY412,12.4
1,E101,7.6
1,CS216,10.5
2,PY205,9.3
2,PY411,11.2
2,PY412,9.6
2,E101,5.8
2,CS216,9.899999999999999


Finally, take a look at the reference page on [Core Functions](https://www.sqlite.org/lang_corefunc.html) and the reference page on [Expressions](https://www.sqlite.org/lang_expr.html). The core functions can be used in any expression, and the expression syntax has other features. Here are two examples.

Using a CASE expression to transform numerical grades to letters:

In [20]:
%%sql SELECT studentid, course,
       CASE WHEN grade<0.66 THEN 'F'
            WHEN grade<1.66 THEN 'D'
            WHEN grade<2.66 THEN 'C'
            WHEN grade<3.66 THEN 'B'
            ELSE 'A' END
                AS lettergrade
    FROM grades;

 * sqlite:///students_grades.db
Done.


studentid,course,lettergrade
1,PY205,A
1,PY411,B
1,PY412,B
1,E101,A
1,CS216,B
2,PY205,B
2,PY411,B
2,PY412,C
2,E101,B
2,CS216,B


Using a LIKE expression in the WHERE clause to match all Physics classes. LIKE expressions do pattern matching using the percent sign as a wildcard.

In [21]:
%sql SELECT * FROM grades WHERE course LIKE "PY%";

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,PY411,3.2,4
1,PY412,3.1,4
2,PY205,3.1,3
2,PY411,2.8,4
2,PY412,2.4,4


Now you know how to do simple selects and queries on a table, filtering results and doing some simple computations. Take some time to play with various expressions and queries by your own!   

#### Grouping and Aggregate Queries
So you can now select rows from the table and compute expressions per-row, but what if you want to compute a student's GPA from all their courses? Because each (student,course) pair is its own row, we can't computer a GPA because that involves data from more than one row.

SQL provides a grouping mechanism that allows you to do aggregate computations on each group. This is done with the GROUP BY clause, which goes after the FROM and WHERE clauses.

The GROUP BY clause takes one or more expressions, separated by commas, that define the groups. Typically, these expressions will be column names but you can compute values with expressions, too.

SQL will then take each row and assign it a group based on the GROUP BY expression(s). **The results of the query will be one row per group.** From the [SQLite docs](https://www.sqlite.org/lang_select.html#resultset):
> [...] each of the expressions specified as part of the GROUP BY clause is evaluated for each row of the dataset. Each row is then assigned to a "group" based on the results; rows for which the results of evaluating the GROUP BY expressions are the same get assigned to the same group.

Using a GROUP BY clause makes the query into an *aggregate query*. In your SELECT clause, you can then use one or more [aggregate functions](https://www.sqlite.org/lang_aggfunc.html). The aggregate functions operate over each row in the group. So, for example, you can compute the average grade per student like this:

In [23]:
%sql SELECT studentid, AVG(grade) AS avg_grade FROM grades GROUP BY studentid;

 * sqlite:///students_grades.db
Done.


studentid,avg_grade
1,3.4799999999999995
2,2.9


Now we know enough to use grouping to compute each student's GPA:

In [24]:
%sql SELECT studentid, SUM(grade*credits)/SUM(credits) AS GPA FROM grades GROUP BY studentid;

 * sqlite:///students_grades.db
Done.


studentid,GPA
1,3.41875
2,2.8625


This takes the total gradepoints of each group and divides them by the total credits in that group. Since the grouping is by studentid, each student is a group, and each group contains the rows for that student. So any aggregate functions operate on the rows for each student independently.

If you combine the WHERE clause with an aggregate query, the WHERE clause filters the input rows before grouping. So here is the GPA of all students just for their Physics classes:

In [25]:
%sql SELECT * FROM grades WHERE course LIKE "PY%";

 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits
1,PY205,3.8,3
1,PY411,3.2,4
1,PY412,3.1,4
2,PY205,3.1,3
2,PY411,2.8,4
2,PY412,2.4,4


If you add a HAVING clause, this filters the groups. It's essentially like the WHERE clause except it applies *after* grouping instead of before. You can use aggregate expressions in the HAVING clause.

This selects students that have a GPA greater than 3. Notice how we don't actually select the GPA into the results, we just get the studentids, and calculate the GPA in the HAVING clause.

In [28]:
%sql SELECT studentid FROM grades GROUP BY studentid HAVING SUM(grade*credits)/SUM(credits) >= 3;

 * sqlite:///students_grades.db
Done.


studentid
1


You can also refer to a column name in other clauses. So if you *do* want the GPA in the results, to avoid having to duplicate the expression in both clauses, give it a name and refer to it by name:

In [38]:
%%sql SELECT studentid, SUM(grade*credits)/SUM(credits) AS GPA 
    FROM grades
    GROUP BY studentid
    HAVING GPA >= 3;

 * sqlite:///students_grades.db
Done.


studentid,GPA
1,3.41875


#### Two more notes on Aggregate Queries

**1.** if you use an *aggregate function* but don't use a GROUP BY clause, the query is still an aggregate query, but with one large group containing every row.

This is the total GPA of all students, which may or may not be meaningful.

In [29]:
%sql SELECT SUM(grade*credits)/SUM(credits) AS GPA FROM grades;

 * sqlite:///students_grades.db
Done.


GPA
3.140625


**2.** If you select on a column that is neither an aggregate function nor a column named in the GROUP BY, you're essentially asking about a column for which there may be multiple results. This is allowed, but since the results only give one row per group, SQLite will pick an arbitrary row within the group to return, which is usually not meaningful.

From the [SQLite docs](https://www.sqlite.org/lang_select.html#resultset):
> Each expression in the result-set is then evaluated once for each group of rows. If the expression is an aggregate expression, it is evaluated across all rows in the group. Otherwise, it is evaluated against a single arbitrarily chosen row from within the group. If there is more than one non-aggregate expression in the result-set, then all such expressions are evaluated for the same row.


For example, this query groups by students and also selects the course column. But since each student takes more than one course, this will return an arbitrary course for each student.

In [30]:
%sql SELECT studentid, course FROM grades

 * sqlite:///students_grades.db
Done.


studentid,course
1,PY205
1,PY411
1,PY412
1,E101
1,CS216
2,PY205
2,PY411
2,PY412
2,E101
2,CS216


In [31]:
%sql SELECT studentid, course FROM grades GROUP BY studentid;

 * sqlite:///students_grades.db
Done.


studentid,course
1,PY205
2,PY205


### Joins

Joins are one of the most powerful features of SQL, but are also tricky to master. They allow you to join two or more tables of data together. Before we get started, we'll need another table of data.

Let's import the contents of demographics.csv into our database, this one with just 3 rows:

```
studentid,gender,age
1,m,22
2,f,19
3,m,20
```

In [32]:
# Load the CSV data
csv_file_path = 'demographics.csv'
data = pd.read_csv(csv_file_path)

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('students_demographics.db')

# Save the DataFrame to an SQLite table
data.to_sql('demographics', conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data imported into the SQLite database successfully!")

Data imported into the SQLite database successfully!


In [33]:
%sql sqlite:///students_demographics.db

In [34]:
%sql SELECT * FROM demographics;

 * sqlite:///students_demographics.db
   sqlite:///students_grades.db
Done.


studentid,gender,age
1,m,22
2,f,19
3,m,20


Now we can join the tables together. At its core, a join is simply a cartesian-product of the rows in two input tables. So for every row in one input table, every row in the second table will be returned.

From the [SQLite docs](https://www.sqlite.org/lang_select.html#fromclause):
> There is a row in the cartesian product dataset formed by combining each unique combination of a row from the left-hand and right-hand datasets. In other words, if the left-hand dataset consists of Nleft rows of Mleft columns, and the right-hand dataset of Nright rows of Mright columns, then the cartesian product is a dataset of Nleft×Nright rows, each containing Mleft+Mright columns.

The following is a nonsensical query demonstrating this, but it's an important mental model to have for how joins work. Interacting with more that one database is a bit complicated using our kernel, so we go back to our previous method on how to connect to a database.

In [35]:
# Connect to the first database (students_grades.db)
conn = sqlite3.connect('students_grades.db')
cursor = conn.cursor()

# Attach the second database (demographics.db)
cursor.execute("ATTACH DATABASE 'students_demographics.db' AS demo_db;")

# Perform the join query between the grades table and the demographics table
query = """
SELECT g.*, d.*
FROM grades g
JOIN demo_db.demographics d
ON g.studentid = d.studentid;
"""
cursor.execute(query)

# Fetch all results from the query
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

# Detach the attached database and close the connection
cursor.execute("DETACH DATABASE demo_db;")
conn.close()

(1, 'PY205', 3.8, 3, 1, 'm', 22)
(1, 'PY411', 3.2, 4, 1, 'm', 22)
(1, 'PY412', 3.1, 4, 1, 'm', 22)
(1, 'E101', 3.8, 2, 1, 'm', 22)
(1, 'CS216', 3.5, 3, 1, 'm', 22)
(2, 'PY205', 3.1, 3, 2, 'f', 19)
(2, 'PY411', 2.8, 4, 2, 'f', 19)
(2, 'PY412', 2.4, 4, 2, 'f', 19)
(2, 'E101', 2.9, 2, 2, 'f', 19)
(2, 'CS216', 3.3, 3, 2, 'f', 19)


In [83]:
%sql sqlite:///students_grades.db

# Now attach the second database (demographics.db)
%sql ATTACH DATABASE 'students_demographics.db' AS demo_db;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


[]

In [84]:
# Query to join grades from students_grades.db and demographics from demo_db
%sql SELECT g.*, d.* FROM grades g JOIN demo_db.demographics d ON g.studentid = d.studentid;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


studentid,course,grade,credits,studentid_1,gender,age
1,PY205,3.8,3,1,m,22
1,PY411,3.2,4,1,m,22
1,PY412,3.1,4,1,m,22
1,E101,3.8,2,1,m,22
1,CS216,3.5,3,1,m,22
2,PY205,3.1,3,2,f,19
2,PY411,2.8,4,2,f,19
2,PY412,2.4,4,2,f,19
2,E101,2.9,2,2,f,19
2,CS216,3.3,3,2,f,19


In [87]:
%sql DETACH DATABASE demo_db;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


[]

In [89]:
%sql sqlite:///students_grades.db

# Now attach the second database (demographics.db)
%sql ATTACH DATABASE 'students_demographics.db' AS demo_db;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


[]

#### Joins and grouping
Joins work with groups just as you'd expect. All joins happen before grouping, so you can get a student's demographic information along with their GPA in one query:

In [90]:
%%sql SELECT g.studentid, SUM(g.grade*g.credits)/SUM(g.credits) AS GPA, d.gender, d.age
    FROM grades g JOIN demo_db.demographics d ON g.studentid=d.studentid
    GROUP BY g.studentid;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


studentid,GPA,gender,age
1,3.41875,m,22
2,2.8625,f,19


#### Outer joins

Left Outer joins are the other join type supported by SQLite. It's not as useful but it's still good to know about it.

An inner join as seen above is easy to think about as a simple filter on the cartesian product of two tables. A left outer join, however, is useful when you want to return *every* row on the left-hand table *whether or not* a matching row is found on the right-hand table.

Notice the demographics table we created above had a third student in it, but that student wasn't in any of our results, because he didn't match any rows in the grades table.

Here's how we can get every student, and if they have one, their GPA:

In [91]:
%%sql SELECT d.studentid, SUM(g.grade*g.credits)/sum(g.credits) AS GPA, d.gender, d.age
    FROM demo_db.demographics d LEFT JOIN grades g ON d.studentid=g.studentid
    GROUP BY g.studentid;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


studentid,GPA,gender,age
3,None,m,20
1,3.41875,m,22
2,2.8625,f,19


In [93]:
%sql DETACH DATABASE demo_db;

   sqlite:///students_demographics.db
 * sqlite:///students_grades.db
Done.


[]

And there you have it. Student 3's GPA is left blank because he didn't have any matching courses in the grades table.

## Advanced Example: HR Database
<hr>

Suppose that you are creating a HR app for a firm. You have to create a simple database to store the employees' information. However, you want to apply the DRY principle and avoid redundancy in your code. Here is a example on how to create a database and insert data into it.

Let's first define a class to handle the database connection and the cursor. 

In [92]:
class Employee:
    """A sample Employee class"""

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

    def __repr__(self):
        return "Employee('{}', '{}', {})".format(self.first, self.last, self.pay)

In [105]:
#conn = sqlite3.connect(':memory:')
conn = sqlite3.connect("employees.db")

c = conn.cursor()

c.execute("""create table employees (
            first text,
            last text,
            pay integer
            )""")

In [106]:
def insert_emp(emp):
    """ Insert an employee into the database """
    with conn:
        c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp.first, 'last': emp.last, 'pay': emp.pay})


def get_emps_by_name(lastname):
    """ Get employees by last name """
    c.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
    return c.fetchall()


def update_pay(emp, pay):
    """ Update an employee's pay """
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})


def remove_emp(emp):
    """ Remove an employee from the database """
    with conn:
        c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': emp.first, 'last': emp.last})


Let's add two new emloyees to the database:

In [107]:
emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

In [108]:
insert_emp(emp_1)
insert_emp(emp_2)

In [109]:
%sql sqlite:///employees.db
%sql SELECT * FROM employees;

 * sqlite:///employees.db
   sqlite:///students_demographics.db
   sqlite:///students_grades.db
Done.


first,last,pay
John,Doe,80000
Jane,Doe,90000


Search employees by name Doe in the database:

In [110]:
emps = get_emps_by_name('Doe')
print(emps)

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]


Do some updates in the database:

In [111]:
update_pay(emp_2, 95000)

%sql SELECT * FROM employees;

 * sqlite:///employees.db
   sqlite:///students_demographics.db
   sqlite:///students_grades.db
Done.


first,last,pay
John,Doe,80000
Jane,Doe,95000


In [112]:
remove_emp(emp_1)
emps = get_emps_by_name('Doe')
print(emps)

%sql SELECT * FROM employees;

[('Jane', 'Doe', 95000)]
 * sqlite:///employees.db
   sqlite:///students_demographics.db
   sqlite:///students_grades.db
Done.


first,last,pay
Jane,Doe,95000


In [113]:
conn.close()

# <center>Thank you!<a class="tocSkip"></center>
***